# Internship Placement Selector

This notebook selects internship placements with two non-mixing shifts, using ranked preferences, sex requirements, seat caps, and per-shift lotteries when oversubscribed.

**Files expected**  
- **Students CSV** (`student_path`):  
  Columns (exact order): `"student_name","student_id","sex","shift","rank 1",...,"rank x"`  
  - `sex` in `{male,female}`, `shift` in `{1,2}`
  - Each `"rank i"` contains a site code.
- **Drugstores CSV** (`drugstore_path`):  
  Columns (exact order): `"code","branch","sex_require1","seat1","sex_require2","seat2"`  
  - `sex_require1/2` in `{male,female,both}` (applies to Shift 1 / Shift 2)  
  - `seat1/2` = integer seats for Shift 1 / Shift 2

**Output**  
A CSV named with current datetime prefix + `_output.csv`, columns:  
`"student_name","student_id","rank_result","result"` (same student order as input).

**Selection rules**  
- Process *Shift 1* first, then *Shift 2*.  
- Iterate ranks `1 → x`. For each site & rank & shift: if oversubscribed, run a random lottery within that shift; winners assigned, non-winners move to next rank.  
- Enforce sex requirements for the corresponding shift; if not satisfied, skip that rank choice.  
- After rank `x`, unassigned students get `rank_result=0`, `result="Not selected"`.  
- Print a summary of matches by rank and the count of not selected.


In [17]:
# Changing directory
%cd Users/suparvitp/Programming/internshipPlacementSelectionSystem

[Errno 2] No such file or directory: 'Users/suparvitp/Programming/internshipPlacementSelectionSystem'
/Users/suparvitp/Programming/internshipPlacementSelectionSystem


In [18]:
# =======================
# Cell 0 - Import library
# =======================

import os
from datetime import datetime
import random
import numpy as np
import pandas as pd

In [19]:
# =======================
# Cell 1 - Configuration
# =======================

# Number of ranked choices per student (must match your CSV headers: rank 1 ... rank x)
x = 5

# Students CSV path (columns: student_name, student_id, sex, shift, rank 1 ... rank x)
student_path = "student_path_100.csv"
print(os.path.exists(student_path))

# Drugstores CSV path (columns: code, branch, sex_require1, seat1, sex_require2, seat2)
drugstore_path = "drugstore_path.csv"
print(os.path.exists(drugstore_path))

# Directory to save the output as CSV
output_path = "/"

# Set a seed for reproducible random tie-breaks (None for non-deterministic)
random_seed = 42


True
True


In [20]:
# Display student dataframe
student_df = pd.read_csv(student_path)
student_df

,student_name,student_id,sex,shift,rank 1,rank 2,rank 3,rank 4,rank 5
0,Student001,620001,male,1,PUR38,PUR02,BKK05,FCN67,FCN83
1,Student002,620002,female,1,BTS23,PUR19,PUR21,FCN71,CEN14
2,Student003,620003,male,1,PUR19,FCN32,BKK40,BKK19,CEN01
3,Student004,620004,female,2,FCN77,FCN05,PUR30,CEN18,CEN02
4,Student005,620005,male,1,FCN84,CEN12,BKK01,FCN23,BKK14
...,...,...,...,...,...,...,...,...,...
95,Student096,620096,male,2,FCN11,BKK03,FCN91,FCN49,PUR46
96,Student097,620097,female,2,FCN79,FCN82,PUR25,FCN94,FCN25
97,Student098,620098,male,2,PUR33,CEN01,PUR40,BKK31,BKK39
98,Student099,620099,female,2,BKK11,FCN58,FCN69,HPT04,BKK22


In [21]:
# Display drugstore dataframe
drugstore_df = pd.read_csv(drugstore_path)
drugstore_df

,code,branch,sex_require1,seat1,sex_require2,seat2
0,CEN01,6 สโมสรวัฒนธรรมหญิง,both,2.0,both,2.0
1,CEN02,7 บุญมี ปุรุราชรังสรรค์,both,2.0,NaN,NaN
2,CEN03,19 วงศ์สว่าง,male,1.0,male,1.0
3,CEN04,20 ป้อมปราบศัตรูพ่าย,NaN,NaN,both,2.0
4,CEN05,23 สี่พระยา,male,1.0,male,1.0
...,...,...,...,...,...,...
240,FCN93,คาลเท็กซ์ ธัญบุรี-รังสิต (คลอง3),both,2.0,both,2.0
241,FCN94,เดอะพร้อม ดินแดง,both,1.0,both,1.0
242,FCN95,บิ๊กซี กรีนเวลท์ พระราม 2,both,1.0,both,1.0
243,FCN96,ศิริราช,both,2.0,both,2.0


In [22]:
# =======================
# Cell 2 - Helpers
# =======================

if random_seed is not None:
    random.seed(random_seed)
    np.random.seed(random_seed)

SHIFT_ORDER = ["1", "2"]

def _normalize_sex(v):
    if isinstance(v, str):
        s = v.strip().lower()
        if s in {"male","m"}: return "male"
        if s in {"female","f"}: return "female"
        if s in {"both","any"}: return "both"
    return v

def _normalize_shift(v):
    if pd.isna(v): return ""
    return str(v).strip()

def _check_paths(student_path, drugstore_path, output_path):
    if not os.path.isfile(student_path):
        raise FileNotFoundError(f"students CSV not found: {student_path}")
    if not os.path.isfile(drugstore_path):
        raise FileNotFoundError(f"drugstores CSV not found: {drugstore_path}")
    if not os.path.isdir(output_path):
        raise NotADirectoryError(f"output_path is not a directory: {output_path}")

def _validate_student_columns(df, x):
    expected = ["student_name","student_id","sex","shift"] + [f"rank {i}" for i in range(1, x+1)]
    if list(df.columns)[:len(expected)] != expected:
        raise ValueError(
            "Students CSV must start with columns: "
            + ", ".join(expected)
            + f"\nFound: {list(df.columns)[:len(expected)]}"
        )

def _validate_drugstore_columns(df):
    expected = ["code","branch","sex_require1","seat1","sex_require2","seat2"]
    if list(df.columns)[:len(expected)] != expected:
        raise ValueError(
            "Drugstores CSV must start with columns: "
            + ", ".join(expected)
            + f"\nFound: {list(df.columns)[:len(expected)]}"
        )

def _sex_allowed(req, student_sex):
    if req in (None, "both"): return True
    if req == "male": return student_sex == "male"
    if req == "female": return student_sex == "female"
    return False


In [ ]:
# =======================
# Cell 3 — Load Data & Run Selection
# =======================

# 1) Validate paths and load
_check_paths(student_path, drugstore_path, output_path)
students_raw = pd.read_csv(student_path)
drugstores_raw = pd.read_csv(drugstore_path)

# 2) Validate schemas
_validate_student_columns(students_raw, x)
_validate_drugstore_columns(drugstores_raw)

# 3) Normalize data
students = students_raw.copy()
students["sex"] = students["sex"].apply(_normalize_sex)
students["shift"] = students["shift"].apply(_normalize_shift)
for i in range(1, x+1):
    col = f"rank {i}"
    if col in students.columns:
        students[col] = students[col].astype(str).str.strip()

drugstores = drugstores_raw.copy()
drugstores["code"] = drugstores["code"].astype(str).str.strip()
drugstores["sex_require1"] = drugstores["sex_require1"].apply(_normalize_sex)
drugstores["sex_require2"] = drugstores["sex_require2"].apply(_normalize_sex)

def safe_int(val):
    if pd.isna(val):
        return 0
    if isinstance(val, str):
        v = val.strip()
        if v == '' or v == '-':
            return 0
        try:
            return int(float(v))
        except Exception:
            return 0
    try:
        return int(val)
    except Exception:
        return 0

# 4) Remove duplicated rank

import pandas as pd
from datetime import datetime

def _norm_code(v):
    if pd.isna(v):
        return ""
    return str(v).strip()

removal_logs = [] 

students_clean = students.copy()

for idx, row in students_clean.iterrows():
    sid   = row["student_id"]
    sname = row["student_name"]

    code_ranks = {}
    for k in range(1, x+1):
        code = _norm_code(row.get(f"rank {k}", ""))
        if code:
            code_ranks.setdefault(code, []).append(k)

    # keep lowest rank
    kept = []
    for code, ranks_list in code_ranks.items():
        kept_rank = min(ranks_list)
        kept.append((kept_rank, code))
        # if duplicated rank more than one
        if len(ranks_list) > 1:
            removed_ranks = sorted([r for r in ranks_list if r != kept_rank])
            removal_logs.append({
                "student_id": sid,
                "student_name": sname,
                "code": code,
                "kept_rank": kept_rank,
                "removed_ranks": removed_ranks,
                "removed_count": len(removed_ranks),
            })

    # sort rank
    kept_sorted = [code for _, code in sorted(kept, key=lambda t: t[0])]

    # เติมช่องว่างให้ครบ x
    while len(kept_sorted) < x:
        kept_sorted.append("")

    for k in range(1, x+1):
        students_clean.at[idx, f"rank {k}"] = kept_sorted[k-1]

students = students_clean   # replace df

print("Duplicate ranks cleaned. Remaining ranks are compacted (no gaps).")

if removal_logs:
    removals_df = pd.DataFrame(removal_logs, columns=[
        "student_id", "student_name", "code", "kept_rank", "removed_ranks", "removed_count"
    ])
    print(f"Found {len(removals_df)} duplicate-code incidents across students.")
    display(removals_df.sort_values(["student_id","code"]).reset_index(drop=True))

else:
    print("No duplicate ranks were found.")

# 5) Build site info: per-shift requirements & seat pools (separate)
#    Structure: site_info[code] = {
#        "branch": str,
#        "by_shift": { "1": {"sex_req": ..., "seats_left": int},
#                      "2": {"sex_req": ..., "seats_left": int} }
#    }
site_info = {}
for _, r in drugstores.iterrows():
    code = r["code"]
    site_info[code] = {
        "branch": r["branch"],
        "by_shift": {
            "1": {
                "sex_req": r["sex_require1"],
                "seats_left": safe_int(r["seat1"])
            },
            "2": {
                "sex_req": r["sex_require2"],
                "seats_left": safe_int(r["seat2"])
            },
        }
    }

# 6) Prepare tracking
n_students = len(students)
assigned = pd.Series(False, index=students.index)
rank_result = pd.Series(0, index=students.index, dtype=int)
result = pd.Series("Not selected", index=students.index, dtype=object)
rank_match_counts = {i: 0 for i in range(1, x+1)}

# 7) Process shift by shift (Shift 1 -> Shift 2), ranks 1..x
# For each shift
for shift in SHIFT_ORDER:
    # For each rank
    for r in range(1, x+1):
        rank_col = f"rank {r}"
        # Candidates in this shift, unassigned, with a non-empty code
        mask = ((~assigned) #hasn't yet assigned
        & (students["shift"] == shift)  #in process
        & students[rank_col].notna()    #has seats left in this rank
        & (students[rank_col].astype(str).str.strip() != ""))   #not NaN
        if not mask.any():
            continue
        df_rank = students.loc[mask, ["sex", rank_col]].copy()
        df_rank["code"] = df_rank[rank_col].astype(str).str.strip()

        # Keep only valid site codes
        df_rank = df_rank[df_rank["code"].isin(site_info.keys())]
        if df_rank.empty:
            continue

        # Build applicants per site (respect sex requirement for THIS shift)
        applicants_by_site = {}
        for idx, row in df_rank.iterrows():
            code = row["code"]
            info = site_info[code]["by_shift"].get(shift, None)
            if info is None:
                continue
            if not _sex_allowed(info["sex_req"], row["sex"]):
                continue
            if applicants_by_site.get(code) is None:
                applicants_by_site[code] = []
            applicants_by_site[code].append(idx)

        # Resolve each site: assign all if not oversubscribed; else lottery
        for code, applicants in applicants_by_site.items():
            seats_left = int(site_info[code]["by_shift"][shift]["seats_left"])
            if seats_left <= 0:
                continue
            # Remove any already assigned (safety)
            pool = [i for i in applicants if not assigned.loc[i]]
            if not pool:
                continue
            if len(pool) <= seats_left:
                winners = pool
            else:
                winners = list(np.random.choice(pool, size=seats_left, replace=False))

            # Assign winners
            for w in winners:
                assigned.loc[w] = True
                rank_result.loc[w] = r
                result.loc[w] = code

            # Decrement seats
            site_info[code]["by_shift"][shift]["seats_left"] = max(0, seats_left - len(winners))

            # Count matches for this rank (overall)
            rank_match_counts[r] += len(winners)
print("Selection complete.")

# 8) Build output (same order as students CSV)
output_df = pd.DataFrame({
    "student_name": students["student_name"],
    "student_id": students["student_id"],
    "shift": students["shift"],
    "rank_result": rank_result,
    "result": result
})

# 9) Summary
unmatched = int((rank_result == 0).sum())
print("=== Summary ===")
for r in range(1, x+1):
    print(f"Rank {r}: {rank_match_counts[r]} matched")
print(f"Not selected: {unmatched}")

# 10) Save CSV
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
outfile = os.path.join(output_path, f"{ts}_output.csv")
output_df.to_csv('./'+outfile, index=False)
print(f"Saved to: {outfile}")


Duplicate ranks cleaned. Remaining ranks are compacted (no gaps).
Found 3 duplicate-code incidents across students.


,student_id,student_name,code,kept_rank,removed_ranks,removed_count
0,620029,Student029,FCN42,1,[2],1
1,620034,Student034,FCN81,4,[5],1
2,620035,Student035,HPT04,2,[5],1


=== Summary (Overall) ===
Rank 1: 92 matched
Rank 2: 7 matched
Rank 3: 1 matched
Rank 4: 0 matched
Rank 5: 0 matched
Not selected: 0
Saved to: /20250912_133635_output.csv


In [ ]:
# =======================
# Cell 4 - Verification cell
# =======================
print("=== Verification Start ===")

def _norm_code(s):
    return "" if pd.isna(s) else str(s).strip()

def _norm_shift(s):
    return "" if pd.isna(s) else str(s).strip()

def _norm_sex(s):
    if pd.isna(s): return None
    s = str(s).strip().lower()
    if s in {"male","m"}: return "male"
    if s in {"female","f"}: return "female"
    if s in {"both","any"}: return "both"
    return s

def _sex_allowed(requirement, student_sex):
    req = _norm_sex(requirement)
    sx  = _norm_sex(student_sex)
    if req in (None, "both"): return True
    return req == sx

issues = []

# Normalize fields for robust comparisons
_students = students.copy()
_students["shift"] = _students["shift"].map(_norm_shift)
_students["sex"]   = _students["sex"].map(_norm_sex)
for i in range(1, x+1):
    col = f"rank {i}"
    if col in _students.columns:
        _students[col] = _students[col].map(_norm_code)

_output = output_df.copy()
_output["result"] = _output["result"].map(_norm_code)

_drug = drugstores.copy()
_drug["code"] = _drug["code"].map(_norm_code)
_drug["sex_require1"] = _drug["sex_require1"].map(_norm_sex)
_drug["sex_require2"] = _drug["sex_require2"].map(_norm_sex)

# ---- 1) Counts & IDs ----
print("\n[1] Counts & IDs")
n_students = len(_students)
n_output   = len(_output)
if n_students != n_output:
    issues.append(f"Row count mismatch: students={n_students}, output={n_output}")
print(f"- Row counts: students={n_students}, output={n_output}")

dup_students = _students[_students.duplicated(subset=["student_id"], keep=False)]
if not dup_students.empty:
    issues.append(f"Duplicate student_id in students: {dup_students['student_id'].tolist()}")
    print("- Duplicate student_id in students:")
    display(dup_students)
else:
    print("- No duplicate student_id in students")

dup_output = _output[_output.duplicated(subset=["student_id"], keep=False)]
if not dup_output.empty:
    issues.append(f"Duplicate student_id in output: {dup_output['student_id'].tolist()}")
    print("- Duplicate student_id in output:")
    display(dup_output)
else:
    print("- No duplicate student_id in output")

missing_in_output = set(_students["student_id"]) - set(_output["student_id"])
missing_in_students = set(_output["student_id"]) - set(_students["student_id"])
if missing_in_output:
    issues.append(f"Students missing in output: {len(missing_in_output)}")
    print(f"- Missing in output: {len(missing_in_output)}")
if missing_in_students:
    issues.append(f"Unexpected student_id in output: {len(missing_in_students)}")
    print(f"- Unexpected IDs in output: {len(missing_in_students)}")

# ---- 2) Rank Correctness & in-any-rank ----
print("\n[2] Rank Correctness (and whether result is in any chosen ranks)")
bad_rank_alignment = []
not_in_any_rank = []

for i in range(len(_output)):
    rr  = _output.iloc[i]["rank_result"]
    res = _output.iloc[i]["result"]
    sid = _output.iloc[i]["student_id"]
    sname = _output.iloc[i]["student_name"]

    if (pd.isna(rr)) or (int(rr) < 0) or (int(rr) > x):
        issues.append(f"Invalid rank_result for {sid}: {rr}")
        continue
    rr = int(rr)

    if rr == 0:
        if res and res.lower() != "not selected":
            issues.append(f"{sid} has rank_result=0 but result='{res}'")
        continue

    chosen = _students.iloc[i].get(f"rank {rr}", "")
    if _norm_code(chosen) != res:
        bad_rank_alignment.append((sid, sname, rr, _norm_code(chosen), res))

    chosen_set = {_students.iloc[i].get(f"rank {k}", "") for k in range(1, x+1)}
    chosen_set = {_norm_code(c) for c in chosen_set if str(c).strip() != ""}
    if res not in chosen_set:
        not_in_any_rank.append((sid, sname, res, sorted(list(chosen_set))))

if bad_rank_alignment:
    print("- Assigned differs from code at assigned rank:")
    for sid, sname, rr, expc, gotc in bad_rank_alignment[:20]:
        print(f"  {sid} ({sname}) rank {rr}: expected {expc}, got {gotc}")
    if len(bad_rank_alignment) > 20:
        print(f"  ... ({len(bad_rank_alignment)-20} more)")
    issues.append(f"{len(bad_rank_alignment)} mismatches between assigned rank and code.")

if not_in_any_rank:
    print("- Assigned site not present in any of the student's ranks:")
    for sid, sname, res, chosen in not_in_any_rank[:20]:
        print(f"  {sid} ({sname}): assigned={res}, ranks={chosen}")
    if len(not_in_any_rank) > 20:
        print(f"  ... ({len(not_in_any_rank)-20} more)")
    issues.append(f"{len(not_in_any_rank)} assigned sites not found in any chosen rank.")

# ---- Build capacities & assigned_counts (used by 3 & 4) ----
capacities = defaultdict(lambda: {"1": 0, "2": 0})
sexreq_map  = defaultdict(lambda: {"1": None, "2": None})
for _, r in _drug.iterrows():
    code = r["code"]
    c1 = int(r["seat1"]) if pd.notna(r["seat1"]) else 0
    c2 = int(r["seat2"]) if pd.notna(r["seat2"]) else 0
    capacities[code]["1"] += c1
    capacities[code]["2"] += c2
    sexreq_map[code]["1"] = r["sex_require1"]
    sexreq_map[code]["2"] = r["sex_require2"]

assigned_counts = defaultdict(lambda: {"1": 0, "2": 0})
for i in range(len(_output)):
    rr  = int(_output.iloc[i]["rank_result"])
    res = _output.iloc[i]["result"]
    if rr > 0 and res and res.lower() != "not selected":
        sft = _students.iloc[i]["shift"]
        if sft in {"1","2"}:
            assigned_counts[res][sft] += 1

# ---- 3) Remaining seats by site -> CSV ----
print("\n[3] Remaining Seats by Site (CSV)")

rows = []
for code in capacities.keys():
    for sh in ["1","2"]:
        capv = capacities[code][sh]
        used = assigned_counts[code][sh]
        rem  = capv - used
        rows.append([code, sh, capv, used, rem])

remaining_df = pd.DataFrame(rows, columns=["code","shift","capacity","assigned","remaining"])


remaining_path = os.path.join(output_path, f"{ts}_remaining_seats.csv")
remaining_df.to_csv('./'+remaining_path, index=False)
print(f"- Remaining seats saved to: {remaining_path}")

# ---- 4) Capacity violations ----
print("\n[4] Capacity Violations")
viol = []
for code, cap in capacities.items():
    used1 = assigned_counts[code]["1"]
    used2 = assigned_counts[code]["2"]
    if used1 > cap["1"]:
        viol.append((code, "1", used1, cap["1"]))
    if used2 > cap["2"]:
        viol.append((code, "2", used2, cap["2"]))

if viol:
    print("- Over-capacity detected:")
    for code, sh, used, capv in viol:
        print(f"  site={code}, shift={sh}: assigned={used}, capacity={capv}")
    issues.append(f"{len(viol)} capacity violations.")
else:
    print("- No capacity violations")

# ---- 5) Duplicate ranks within a student ----
print("\n[5] Duplicate Ranks Detected (in a single student's preference list)")
dup_rank_msgs = []
for i in range(len(_students)):
    sid = _students.iloc[i]["student_id"]
    sname = _students.iloc[i]["student_name"]
    prefs = []
    for k in range(1, x+1):
        val = _norm_code(_students.iloc[i].get(f"rank {k}", ""))
        if val:
            prefs.append((k, val))
    by_code = defaultdict(list)
    for k, code in prefs:
        by_code[code].append(k)
    for code, ks in by_code.items():
        if len(ks) > 1:
            dup_rank_msgs.append((sid, sname, code, ks))

if dup_rank_msgs:
    print("- Found duplicate codes within a student's ranks:")
    for sid, sname, code, ks in dup_rank_msgs[:30]:
        print(f"  {sid} ({sname}): code={code} appears in ranks {ks}")
    if len(dup_rank_msgs) > 30:
        print(f"  ... ({len(dup_rank_msgs)-30} more)")
    issues.append(f"{len(dup_rank_msgs)} students have duplicate site codes across ranks.")
else:
    print("- No duplicate ranks within students")

# ---- 6) Sex requirement compliance ----
print("\n[6] Sex Requirement Violations")
sex_viol = []
for i in range(len(_output)):
    rr  = int(_output.iloc[i]["rank_result"])
    res = _output.iloc[i]["result"]
    if rr == 0 or not res or res.lower() == "not selected":
        continue
    sft = _students.iloc[i]["shift"]
    sx  = _students.iloc[i]["sex"]
    req = sexreq_map[res][sft] if sft in {"1","2"} else None
    if not _sex_allowed(req, sx):
        sex_viol.append((
            _students.iloc[i]["student_id"],
            _students.iloc[i]["student_name"],
            sft, res, sx, req
        ))

if sex_viol:
    print("- Sex requirement mismatches:")
    for sid, sname, sft, res, sx, req in sex_viol[:30]:
        print(f"  {sid} ({sname}) shift={sft}, site={res}, student_sex={sx}, required={req}")
    if len(sex_viol) > 30:
        print(f"  ... ({len(sex_viol)-30} more)")
    issues.append(f"{len(sex_viol)} sex requirement violations.")
else:
    print("- No sex requirement violations")

# ---- Final summary ----
print("\n=== Summary ===")
if issues:
    print("Verification found issues:")
    for it in issues:
        print(" -", it)
else:
    print("All verification checks passed.")

print("=== Verification End ===")

=== Verification Start ===

[1] Counts & IDs
- Row counts: students=100, output=100
- No duplicate student_id in students
- No duplicate student_id in output

[2] Rank Correctness (and whether result is in any chosen ranks)

[3] Remaining Seats by Site (CSV)
- Remaining seats saved to: /20250912_133635_remaining_seats.csv

[4] Capacity Violations
- No capacity violations

[5] Duplicate Ranks Detected (in a single student's preference list)
- No duplicate ranks within students

[6] Sex Requirement Violations
- No sex requirement violations

=== Summary ===
All verification checks passed ✅
=== Verification End ===


In [25]:
# =======================
# Cell 5 - Preview df
# =======================
try:
    display(output_df)
except NameError:
    print("Run previous cells first.")


,student_name,student_id,shift,rank_result,result
0,Student001,620001,1,1,PUR38
1,Student002,620002,1,1,BTS23
2,Student003,620003,1,1,PUR19
3,Student004,620004,2,1,FCN77
4,Student005,620005,1,1,FCN84
...,...,...,...,...,...
95,Student096,620096,2,1,FCN11
96,Student097,620097,2,1,FCN79
97,Student098,620098,2,1,PUR33
98,Student099,620099,2,1,BKK11
